In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!python3 -m pip install xmltodict numpy pandas matplotlib mne scikit-learn scipy joblib autoreject tqdm PyWavelets spectrum mock

In [0]:
#%% #*Import Statements
import os
import sys
import xmltodict
import json
import numpy as np
import pandas as pd
import scipy.stats as sp
import matplotlib.pyplot as plt
import mne
from tqdm import tqdm as tqdm
from spectrum import arburg
from autoreject import AutoReject
import mock

In [0]:
#%% #*Machine Dependent Parameters
DATA_DIR = '/content/drive/My Drive/mahnob/Sessions'
WINSIZE = 3 # 3 seconds
NCHAN = 32

In [0]:
#%% #*Function to get data files
def getDataFiles(rootFolder):
  sessionFolders, sessions = list(sorted(os.listdir(rootFolder))), []
  for sessionFolder in sessionFolders:
    _path = os.path.join(DATA_DIR, sessionFolder)
    if os.path.isdir(_path):
      node = {'folder': sessionFolder}
      for subfile in os.listdir(_path):
        if subfile.endswith('.bdf'):
          node['bdf'] = subfile
        elif subfile.endswith('.xml'):
          node['xml'] = subfile
      if 'bdf' in node and 'xml' in node and 'S_Trial' in node['bdf']:
        sessions.append(node)
  return sessions

In [0]:
def coeffVar(a):
    b = a #Extracting the data from the 32 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    k = 0; #For counting the current row no.
    for i in b:
        mean_i = np.mean(i) #Saving the mean of array i
        std_i = np.std(i) #Saving the standard deviation of array i
        output[k] = std_i/mean_i #computing coefficient of variation
        k=k+1
    return np.mean(output)

In [0]:
import heapq
from scipy.signal import argrelextrema


def firstDiff(i):
    b=i    
    out = np.zeros(len(b))    
    for j in range(len(i)):
        out[j] = b[j-1]-b[j]# Obtaining the 1st Diffs
        c=out[1:len(out)]
    return c #returns first diff
  
def slopeReducer(p, func):
    b = p #Extracting the data from the 32 channels
    output = np.zeros(len(b)) #Initializing the output array with zeros
    res = np.zeros(len(b)-1)
    
    k = 0; #For counting the current row no.
    for i in b:
        x=i
        amp_max = i[argrelextrema(x, np.greater)[0]]
        t_max = argrelextrema(x, np.greater)[0]
        amp_min = i[argrelextrema(x, np.less)[0]]
        t_min = argrelextrema(x, np.less)[0]
        t = np.concatenate((t_max,t_min),axis=0)
        t.sort()#sort on the basis of time

        h=0
        amp = np.zeros(len(t))
        res = np.zeros(len(t)-1)
        for l in range(len(t)):
            amp[l]=i[t[l]]
           
        
        amp_diff = firstDiff(amp)
        
        t_diff = firstDiff(t)
        
        for q in range(len(amp_diff)):
            res[q] = amp_diff[q]/t_diff[q]         
        output[k] = func(res) 
        k=k+1
    return np.mean(output)


def slopeMean(p):
    return slopeReducer(p, np.mean)

def slopeVar(p):
    return slopeReducer(p, np.var)

In [0]:
def kurtosis(a):
    b = a # Extracting the data from the 32 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 32)
    k = 0; # For counting the current row no.
    for i in b:
        mean_i = np.mean(i) # Saving the mean of array i
        std_i = np.std(i) # Saving the standard deviation of array i
        t = 0.0
        for j in i:
            t += (pow((j-mean_i)/std_i,4)-3)
        kurtosis_i = t/len(i) # Formula: (1/N)*(summation(x_i-mean)/standard_deviation)^4-3
        output[k] = kurtosis_i # Saving the kurtosis in the array created
        k +=1 # Updating the current row no.
    return np.mean(output)

In [0]:
def secDiffMean(a):
    b = a # Extracting the data of the 32 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 32)
    temp1 = np.zeros(len(b[0])-1) # To store the 1st Diffs
    k = 0; # For counting the current row no.
    for i in b:
        t = 0.0
        for j in range(len(i)-1):
            temp1[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
        for j in range(len(i)-2):
            t += abs(temp1[j+1]-temp1[j]) # Summing the 2nd Diffs
        output[k] = t/(len(i)-2) # Calculating the mean of the 2nd Diffs
        k +=1 # Updating the current row no.
    return np.mean(output)

In [0]:
def secDiffMax(a):
    b = a # Extracting the data from the 32 channels
    output = np.zeros(len(b)) # Initializing the output array with zeros (length = 32)
    temp1 = np.zeros(len(b[0])-1) # To store the 1st Diffs
    k = 0; # For counting the current row no.
    t = 0.0
    for i in b:
        for j in range(len(i)-1):
            temp1[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
        t = temp1[1] - temp1[0]
        for j in range(len(i)-2):
            if abs(temp1[j+1]-temp1[j]) > t :
                t = temp1[j+1]-temp1[j] # Comparing current Diff with the last updated Diff Max

        output[k] = t # Storing the 2nd Diff Max for channel k
        k +=1 # Updating the current row no.
    return np.mean(output)

In [0]:
def skewness(arr):
    data = arr 
    skew_array = np.zeros(len(data)) #Initialinling the array as all 0s
    index = 0; #current cell position in the output array
   
    for i in data:
        skew_array[index]=sp.stats.skew(i,axis=0,bias=True)
        index+=1 #updating the cell position
    return np.mean(skew_array)

In [0]:
def firstDiffMean(arr):
    data = arr 
    diff_mean_array = np.zeros(len(data)) #Initialinling the array as all 0s
    index = 0; #current cell position in the output array
   
    for i in data:
        sum=0.0#initializing the sum at the start of each iteration
        for j in range(len(i)-1):
            sum += abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
           
        diff_mean_array[index]=sum/(len(i)-1)
        index+=1 #updating the cell position
    return np.mean(diff_mean_array)

In [0]:
def firstDiffMax(arr):
    data = arr 
    diff_max_array = np.zeros(len(data)) #Initialinling the array as all 0s
    first_diff = np.zeros(len(data[0])-1)#Initialinling the array as all 0s 
    index = 0; #current cell position in the output array
   
    for i in data:
        max=0.0#initializing at the start of each iteration
        for j in range(len(i)-1):
            first_diff[j] = abs(i[j+1]-i[j]) # Obtaining the 1st Diffs
            if first_diff[j]>max: 
                max=first_diff[j] # finding the maximum of the first differences
        diff_max_array[index]=max
        index+=1 #updating the cell position
    return np.mean(diff_max_array)

In [0]:
def fractalDimension(Z, threshold=0.9):

    # Only for 2d image
    assert(len(Z.shape) == 2)

    # From https://github.com/rougier/numpy-100 (#87)
    def boxcount(Z, k):
        S = np.add.reduceat(
            np.add.reduceat(Z, np.arange(0, Z.shape[0], k), axis=0),
                               np.arange(0, Z.shape[1], k), axis=1)

        # We count non-empty (0) and non-full boxes (k*k)
        return len(np.where((S > 0) & (S < k*k))[0])


    # Transform Z into a binary array
    Z = (Z < threshold)

    # Minimal dimension of image
    p = min(Z.shape)

    # Greatest power of 2 less than or equal to p
    n = 2**np.floor(np.log(p)/np.log(2))

    # Extract the exponent
    n = int(np.log(n)/np.log(2))

    # Build successive box sizes (from 2**n down to 2**1)
    sizes = 2**np.arange(n, 1, -1)

    # Actual box counting with decreasing size
    counts = []
    for size in sizes:
        counts.append(boxcount(Z, size))

    # Fit the successive log(sizes) with log (counts)
    coeffs = np.polyfit(np.log(sizes), np.log(counts), 1)
    return -coeffs[0]

In [0]:
#autogressiveModelParametersBurg

def autogressiveModelParametersBurg(labels):
    feature = []
    a1=[]
    a2=[]
    a3=[]
    model_order = 3
    for i in range(32):
        AR, rho, ref = arburg(labels[i], model_order)
        feature.append(AR);
    for i in range(32):
        a1.append(feature[i][0])
        a2.append(feature[i][1])
        a3.append(feature[i][2])
        
    
    ar1=np.mean(a1).real
    ar2=np.mean(a2).real 
    ar3=np.mean(a3).real
    
    br1=np.min(a1).real
    br2=np.min(a2).real 
    br3=np.min(a3).real
    
    cr1=np.max(a1).real
    cr2=np.max(a2).real 
    cr3=np.max(a3).real
    return ar1,ar2,ar3,br1,br2,br3,cr1,cr2,cr3

In [0]:
#hjorth(3 feature)
# function for hjorth 
def hjorth(input):
    realinput = input
    hjorth_activity = np.zeros(len(realinput))
    hjorth_mobility = np.zeros(len(realinput))
    hjorth_diffmobility = np.zeros(len(realinput))
    hjorth_complexity = np.zeros(len(realinput))
    diff_input = np.diff(realinput)
    diff_diffinput = np.diff(diff_input)
    k = 0
    for j in realinput:
        hjorth_activity[k] = np.var(j)
        hjorth_mobility[k] = np.sqrt(np.var(diff_input[k])/hjorth_activity[k])
        hjorth_diffmobility[k] = np.sqrt(np.var(diff_diffinput[k])/np.var(diff_input[k]))
        hjorth_complexity[k] = hjorth_diffmobility[k]/hjorth_mobility[k]
        k = k+1
    return np.mean(hjorth_activity), np.mean(hjorth_mobility), np.mean(hjorth_complexity) #returning hjorth activity, hjorth mobility , hjorth complexity

In [0]:
#maxPwelch

from scipy import signal

def maxPwelch(data_win,Fs):
 
    
    BandF = [0.1, 3, 7, 12, 30]
    PMax = np.zeros([32,(len(BandF)-1)]);
    
    for j in range(32):
        f,Psd = signal.welch(data_win[j,:], Fs)
        
        for i in range(len(BandF)-1):
            fr = np.where((f>BandF[i]) & (f<=BandF[i+1]))
            PMax[j,i] = np.max(Psd[fr])
    
    return np.mean(PMax[:,0]),np.mean(PMax[:,1]),np.mean(PMax[:,2]),np.mean(PMax[:,3])

In [0]:
#wavelet_features

import pywt
def waveletFeatures(epoch,channels):
    cA_values = []
    cD_values = []
    cA_mean = []
    cA_std = []
    cA_Energy =[]
    cD_mean = []
    cD_std = []
    cD_Energy = []
    Entropy_D = []
    Entropy_A = []
    wfeatures = []
    for i in range(channels):
        cA,cD=pywt.dwt(epoch[i,:],'coif1')
        cA_values.append(cA)
        cD_values.append(cD)		#calculating the coefficients of wavelet transform.
    for x in range(channels):   
        cA_mean.append(np.mean(cA_values[x]))
        #wfeatures.append(np.mean(cA_values[x]))
        
        cA_std.append(abs(np.std(cA_values[x])))
        #wfeatures.append(abs(np.std(cA_values[x])))
        
        cA_Energy.append(abs(np.sum(np.square(cA_values[x]))))
        #wfeatures.append(abs(np.sum(np.square(cA_values[x]))))
        
        cD_mean.append(np.mean(cD_values[x]))		# mean and standard deviation values of coefficents of each channel is stored .
        #wfeatures.append(np.mean(cD_values[x]))

        cD_std.append(abs(np.std(cD_values[x])))
        #wfeatures.append(abs(np.std(cD_values[x])))
        
        cD_Energy.append(abs(np.sum(np.square(cD_values[x]))))
        #wfeatures.append(abs(np.sum(np.square(cD_values[x]))))
        
        Entropy_D.append(abs(np.sum(np.square(cD_values[x]) * np.log(np.square(cD_values[x])))))
        #wfeatures.append(abs(np.sum(np.square(cD_values[x]) * np.log(np.square(cD_values[x])))))
        
        Entropy_A.append(abs(np.sum(np.square(cA_values[x]) * np.log(np.square(cA_values[x]))))) 
        #wfeatures.append(abs(np.sum(np.square(cA_values[x]) * np.log(np.square(cA_values[x])))))
        
    a1=np.mean(cA_mean)
    b1=np.mean(cA_std) 
    c1=np.mean(cA_Energy)  
    d1=np.mean(cD_mean)
    e1=np.mean(cD_std)  
    f1=np.mean(cD_Energy)
    g1=np.mean(Entropy_D)
    h1=np.mean(Entropy_A)
    
    a2=np.min(cA_mean)
    b2=np.min(cA_std) 
    c2=np.min(cA_Energy)  
    d2=np.min(cD_mean)
    e2=np.min(cD_std)  
    f2=np.min(cD_Energy)
    g2=np.min(Entropy_D)
    h2=np.min(Entropy_A)
    
    a3=np.max(cA_mean)
    b3=np.max(cA_std) 
    c3=np.max(cA_Energy)  
    d3=np.max(cD_mean)
    e3=np.max(cD_std)  
    f3=np.max(cD_Energy)
    g3=np.max(Entropy_D)
    h3=np.max(Entropy_A)
    return a1,a2,a3,b1,b2,b3,c1,c2,c3,d1,d2,d3,e1,e2,e3,f1,f2,f3,g1,g2,g3,h1,h2,h3

In [0]:
#%% #*The MahnobEEG class
class MahnobEEG:

  def __init__(self, sessionInfo, rootFolder):
    self.sessionInfo, self.rootFolder = sessionInfo, rootFolder
    self.metafile = '{}/{}/{}'.format(self.rootFolder,
                                      self.sessionInfo['folder'],
                                      self.sessionInfo['xml'])
    self.eegfile = '{}/{}/{}'.format(self.rootFolder,
                                     self.sessionInfo['folder'],
                                     self.sessionInfo['bdf'])
    self.featureFile = '{}/{}/features.pkl'.format(self.rootFolder, self.sessionInfo['folder'])
    self.channels = []

  def extractMetadata(self):
    emodims = [
        '@feltArsl', '@feltCtrl', '@feltEmo', '@feltPred', '@feltVlnc',
        '@isStim'
    ]
    # *Extract metadata into meta
    temp = None
    with open(self.metafile) as f:
      temp = xmltodict.parse('\n'.join(f.readlines()))
    temp = json.loads(json.dumps(temp))['session']
    metadata = {
        'subjectid': temp['subject']['@id'],
        'results': {k[1:]: int(temp[k]) for k in emodims},
        'media': {
            'name': temp['@mediaFile'],
            'durationSec': float(temp['@cutLenSec'])
        }
    }
    self.metadata = metadata
    
  def extractBDF(self):
    stdout_old, stderr_old = sys.stdout, sys.stderr
    sys.stdout, sys.stderr = mock.MagicMock(), mock.MagicMock()
    
    raw = mne.io.read_raw_bdf(self.eegfile, preload=True, stim_channel='Status')
    t20 = mne.channels.make_standard_montage(kind='biosemi32')
    raw.set_montage(t20, raise_if_subset=False)
    events = mne.find_events(raw, stim_channel='Status')
    start_samp, end_samp = events[0][0] + 1, events[1][0] - 1
    raw.crop(raw.times[start_samp], raw.times[end_samp])
    self.nchan = 32
    ch2idx = dict(map(lambda x: (x[1], x[0]), list(enumerate(raw.ch_names[:self.nchan]))))
    raw.pick_channels(raw.ch_names[:self.nchan])
    self.ch2idx = dict(map(lambda x: (x[1], x[0]), list(enumerate(raw.ch_names[:self.nchan]))))
    self.df = raw.to_data_frame().rename(columns=ch2idx).T
    self.nda = self.df.to_numpy()
    self.raw = raw
    self.nsamp = self.nda.shape[1]
    self.sfreq = raw.info['sfreq']
    self.samp_step = int(self.sfreq * WINSIZE)
    
    sys.stdout, sys.stderr = stdout_old, stderr_old

  def extractFeatures(self):
    samp_step = self.samp_step
    features = pd.DataFrame() 
    columns = [
        'coeffVar', 'AR1_mean', 'AR1_min', 'AR1_max', 'AR2_mean', 'AR2_min', 'AR2_max',
        'AR3_mean', 'AR3_min', 'AR3_max', 'hjorthActivity', 'hjorthMobility',
        'hjorthComplexity', 'PMax1', 'PMax2', 'PMax3', 'PMax4', 'PRatio1', 'PRatio2',
        'PRatio3', 'PRatio4', 'cA_mean_mean', 'cA_mean_min', 'cA_mean_max', 'cA_std_mean',
        'cA_std_min', 'cA_std_max', 'cA_Energy_mean', 'cA_Energy_min', 'cA_Energy_max',
        'cD_mean_mean', 'cD_mean_min', 'cD_mean_max', 'cD_std_mean', 'cD_std_min',
        'cD_std_max', 'cD_Energy_mean', 'cD_Energy_min', 'cD_Energy_max',
        'Entropy_D_mean', 'Entropy_D_min', 'Entropy_D_max', 'Entropy_A_mean',
        'Entropy_A_min', 'Entropy_A_max'
    ]

    for i in np.arange(0, self.nsamp, samp_step):
      featDict = {}
      df = pd.DataFrame(columns=columns)
      epoch = self.nda[:, i:i + samp_step]
      if epoch.shape[1] < self.sfreq:
        continue
      try:
        featDict['coeffVar'] = coeffVar(epoch)
        featDict['fractalDimension'] = fractalDimension(epoch)
        featDict['kurtosis'] = kurtosis(epoch)
        featDict['secDiffMean'] = secDiffMean(epoch)
        featDict['secDiffMax'] = secDiffMax(epoch)
        featDict['skewness'] = skewness(epoch)
        featDict['firstDiffMean'] = firstDiffMean(epoch)
        featDict['firstDiffMax'] = firstDiffMax(epoch)

        a11, a21, a31, a12, a22, a32, a13, a23, a33 = autogressiveModelParametersBurg(
            epoch)
        featDict['AR1_mean'], featDict['AR1_min'], featDict['AR1_max'] = a11, a12, a13
        featDict['AR2_mean'], featDict['AR2_min'], featDict['AR2_max'] = a21, a22, a23
        featDict['AR3_mean'], featDict['AR3_min'], featDict['AR3_max'] = a31, a32, a33

        featDict['hjorthActivity'], featDict['hjorthMobility'], featDict[
            'hjorthComplexity'] = hjorth(epoch)

        a, b, c, d = maxPwelch(epoch, self.sfreq)
        featDict['PMax1'], featDict['PMax2'], featDict['PMax3'], featDict['PMax4'] = a, b, c, d
        featDict['PRatio1'], featDict['PRatio2'], featDict['PRatio3'], featDict[
            'PRatio4'] = a / b, a / c, b / d, (a + b) / c

        a1, a2, a3, b1, b2, b3, c1, c2, c3, d1, d2, d3, e1, e2, e3, f1, f2, f3, g1, g2, g3, h1, h2, h3 = waveletFeatures(
            epoch, self.nchan)
        featDict['cA_mean_mean'], featDict['cA_mean_min'], featDict['cA_mean_max'] = a1, a2, a3
        featDict['cA_std_mean'], featDict['cA_std_min'], featDict['cA_std_max'] = b1, b2, b3
        featDict['cA_Energy_mean'], featDict['cA_Energy_min'], featDict[
            'cA_Energy_max'] = c1, c2, c3

        featDict['cD_mean_mean'], featDict['cD_mean_min'], featDict['cD_mean_max'] = d1, d2, d3
        featDict['cD_std_mean'], featDict['cD_std_min'], featDict['cD_std_max'] = e1, e2, e3
        featDict['cD_Energy_mean'], featDict['cD_Energy_min'], featDict[
            'cD_Energy_max'] = f1, f2, f3

        featDict['Entropy_D_mean'], featDict['Entropy_D_min'], featDict[
            'Entropy_D_max'] = g1, g2, g3
        featDict['Entropy_A_mean'], featDict['Entropy_A_min'], featDict[
            'Entropy_A_max'] = h1, h2, h3

        for column in columns:
          df[column] = [featDict[column]]
        features = pd.concat([features, df])
      except Exception as err:
        raise err
    
    # target
    features['valence'] = self.metadata['results']['feltVlnc']
    features['arousal'] = self.metadata['results']['feltArsl']
    features['control'] = self.metadata['results']['feltCtrl']
    features['prediction'] = self.metadata['results']['feltPred']
    self.features = features
  
  def saveFeatures(self):
    self.features.to_pickle(self.featureFile)          

In [0]:
sessions = getDataFiles(DATA_DIR)

In [34]:
for session in tqdm(sessions):
  pt = MahnobEEG(session, DATA_DIR)
  pt.extractMetadata()
  pt.extractBDF()
  pt.extractFeatures()
  pt.saveFeatures()



  0%|          | 0/546 [00:00<?, ?it/s]

  0%|          | 1/546 [00:10<1:34:47, 10.44s/it]

  0%|          | 2/546 [00:24<1:43:10, 11.38s/it]

  1%|          | 3/546 [00:34<1:40:17, 11.08s/it]

  1%|          | 4/546 [00:43<1:35:42, 10.59s/it]

  1%|          | 5/546 [01:02<1:56:43, 12.95s/it]

  1%|          | 6/546 [01:19<2:07:38, 14.18s/it]

  1%|▏         | 7/546 [01:36<2:16:09, 15.16s/it]

  1%|▏         | 8/546 [01:44<1:56:49, 13.03s/it]

  2%|▏         | 9/546 [01:58<1:58:12, 13.21s/it]

  2%|▏         | 10/546 [02:14<2:05:09, 14.01s/it]

  2%|▏         | 11/546 [02:21<1:46:39, 11.96s/it]

  2%|▏         | 12/546 [02:37<1:58:00, 13.26s/it]

  2%|▏         | 13/546 [02:50<1:57:04, 13.18s/it]

  3%|▎         | 14/546 [03:12<2:19:34, 15.74s/it]

  3%|▎         | 15/546 [03:29<2:21:59, 16.04s/it]

  3%|▎         | 16/546 [03:49<2:33:58, 17.43s/it]

  3%|▎         | 17/546 [04:06<2:32:21, 17.28s/it]

  3%|▎         | 18/546 [04:25<2:35:41, 17.69s/it]

  3%|▎         | 19/546 [04:42

In [35]:
inp = pd.DataFrame()
for session in tqdm(sessions):
  each = pd.read_pickle(os.path.join(DATA_DIR, session['folder'], 'features.pkl'))
  inp = pd.concat([inp, each])



  0%|          | 0/546 [00:00<?, ?it/s]

  6%|▌         | 32/546 [00:00<00:01, 314.16it/s]

 11%|█         | 61/546 [00:00<00:01, 305.39it/s]

 17%|█▋        | 91/546 [00:00<00:01, 303.27it/s]

 23%|██▎       | 123/546 [00:00<00:01, 306.96it/s]

 27%|██▋       | 148/546 [00:00<00:01, 282.49it/s]

 32%|███▏      | 172/546 [00:00<00:01, 260.26it/s]

 36%|███▌      | 197/546 [00:00<00:01, 254.92it/s]

 41%|████      | 223/546 [00:00<00:01, 255.52it/s]

 45%|████▌     | 248/546 [00:00<00:01, 252.64it/s]

 50%|█████     | 273/546 [00:01<00:01, 249.36it/s]

 55%|█████▍    | 298/546 [00:01<00:01, 243.51it/s]

 59%|█████▉    | 322/546 [00:01<00:00, 233.85it/s]

 63%|██████▎   | 346/546 [00:01<00:00, 226.07it/s]

 68%|██████▊   | 372/546 [00:01<00:00, 234.26it/s]

 73%|███████▎  | 396/546 [00:01<00:00, 232.89it/s]

 77%|███████▋  | 420/546 [00:01<00:00, 231.33it/s]

 81%|████████▏ | 444/546 [00:01<00:00, 227.72it/s]

 86%|████████▌ | 467/546 [00:01<00:00, 223.39it/s]

 90%|████████▉ | 490/546

In [0]:
inp.to_pickle(os.path.join(DATA_DIR, 'data_45.pkl'))